## ETL_MVP_15Gifts_O2 

(File 1 of 2. Main Processing file)

In [86]:
#
# # Ensure we have the needed (third-party) packages installed
# 
# %pip install --quiet requests
# %pip install --quiet beautifulsoup4
# %pip install --quiet pandas
# %pip install --quiet seaborn
# %pip install --quiet matplotlib
# %pip install --quiet tabulate
# %pip install --quiet selenium

In [87]:
#
# # Access my ETL functions
# 
%run ETL_func_defs.ipynb

****************************** ETL functions def file version v0.1.1 ******************************



In [88]:
#
# # Core user-defined variables
# 

# Might increase potential for cross-client re-use and (some)page changes
urlBase = "https://www.o2.co.uk"
urlSub = "/shop/phones#sort=content.sorting.featured&page=" # page=1+ added later

# client_colour = '#000068' # could used to identify in reporting (this one for O2)

# Define selectors for the scrape 
main_content_selector = ".device-tile-wrapper" # .device-info-content
brand_selector = '[data-qa-device-title] [data-qa-device-brand]'
model_family_selector = '[data-qa-device-title] [data-qa-device-modelfamily]'
star_rating_selector = ".star-rating__stars"
rating_value_selector = ".device-rating__text [itemprop='ratingValue']"
upfront_price_selector = '.upfront-cost-details .headline'
upfront_pence_selector = '.upfront-cost-details .pence'
monthly_price_selector = '.monthly-cost-details .costVal'
monthly_pence_selector = '.monthly-cost-details .costPence'

num_pages = 7 # page search depth (Reduce if testing process)


In [89]:
#
# # IS the site up? No point running anything further if not! 
#
response_time = check_site_status(urlBase, max_response_time=4)
if response_time:
    print('{} is up and available (response time: {:.2f} seconds)'.format(urlBase, response_time))
    
    # run it again for the page(site could be up, but page missing? )
    response_time = check_site_status(urlBase+urlSub)
    print('{} is up and available (response time: {:.2f} seconds)'.format(urlBase+urlSub, response_time))
else:
    # Stop any further code below processing
    sys.exit()

https://www.o2.co.uk is up and available (response time: 0.14 seconds)
https://www.o2.co.uk/shop/phones#sort=content.sorting.featured&page= is up and available (response time: 0.05 seconds)


In [90]:
#
# # Extract the page data
#

phone_details_list = extract_phone_details(
    urlBase+urlSub,
    main_content_selector,
    brand_selector,
    model_family_selector,
    star_rating_selector,
    rating_value_selector,
    num_pages,
    upfront_price_selector,
    upfront_pence_selector,
    monthly_price_selector,
    monthly_pence_selector
)



# Visual checks/ref
# Print the extracted phone data details
x = 5  # Limit just to keep the output cleaner in notebook
for phone_details in itertools.islice(phone_details_list, x):
    print(phone_details)

In [ ]:
#
# # (1)Pipeline reporting - Identify outliers
#

# Is there anything that we don't expect/want to see in data coming through pipeline?

# Set some (example) default expectations
max_rating_value=5
min_rating_value=2
max_model_family_length=21

outlier_phone_details = identify_outliers(  phone_details_list, 
                                            max_rating_value, 
                                            min_rating_value, 
                                            max_model_family_length 
                                            )


# Are the prices within a logical range(sample/example values selected)
pound_val_keys = ['Monthly Price', 'Upfront Price' ]
pence_val_keys = ['Monthly Pence','Upfront Pence']

outlier_price_details = identify_price_outliers(phone_details_list, pound_val_keys, pence_val_keys)

# output of outliers handled/shown later

In [ ]:
# #
# # # (2)Pipeline reporting - Monitor/optimise process
# #
# import timeit # specific to this, so not in function defs

# avg_time = measure_extract_phone_details_time(
#     urlBase,
#     urlSub,
#     main_content_selector,
#     brand_selector,
#     model_family_selector,
#     star_rating_selector,
#     rating_value_selector,
#     num_pages,
#     upfront_price_selector,
#     upfront_pence_selector,
#     monthly_price_selector,
#     monthly_pence_selector
# )

# print('Average execution time: {:.2f} seconds'.format(avg_time))

In [ ]:
#
# # Cleaning
#

# Visual checks/ref
# Print the extracted phone data details
x = 3  # Just to keep the output cleaner in notebook
for phone_details in itertools.islice(phone_details_list, x):
    print(phone_details)



# check/remove unexpected chars/ obv typos
unwanted_chars = ['%', '?', '%', '$', "'", '#', ';', ',','-', '*']
phone_details_list = remove_dodgy_chars(phone_details_list, unwanted_chars)

# remove lead/trailing spaces (and doubles+)
phone_details_list = clean_spacing_phone_details(phone_details_list)

# flatten phone_details_list strings to enforce consistency
phone_details_list = lowercase_phone_details(phone_details_list)

# (re-)enforce proper/title caps (but also handle hyphens and 's properly)
phone_details_list = capitalise_phone_details(phone_details_list)

# remove dirty vals in currency fields, ensure floats
currency_keys_list = ['Upfront Price','Upfront Pence', 'Monthly Price', 'Monthly Pence']
phone_details_list = clean_pricing(phone_details_list, currency_keys_list)


# Remove dups (has to be done LAST, as above cleaning increases chances of matches)
de_dup_keys_list = ['Brand', 'Model Family']
phone_details_list = remove_duplicates(phone_details_list, de_dup_keys_list)




# Visual checks/ref
# Print the extracted phone data details
x = 3  # Just to keep the output cleaner in notebook
for phone_details in itertools.islice(phone_details_list, x):
    print(phone_details)

In [ ]:
#
# # Ensure consistency
#

# Set required fields to numeric values & make more customer friendly (rounding/set places)
phone_details_list = convert_to_float(phone_details_list, 'Rating Value', 2)


In [ ]:
#
# # Enrichment / manipulation
#

#
# # Combine the cleaned price/pence values into single 'useful' val.
data_to_combine = ['Monthly Price', 'Monthly Pence']
new_total_key = 'Monthly Total Price'
phone_details_list = combine_currency_values(phone_details_list, data_to_combine, new_total_key)


data_to_combine = ['Upfront Price', 'Upfront Pence']
new_total_key = 'Upfront Total Price'
phone_details_list = combine_currency_values(phone_details_list, data_to_combine, new_total_key)


# Visual checks/ref
# Print the extracted phone data details
x = 3  # Just to keep the output cleaner in notebook
for phone_details in itertools.islice(phone_details_list, x):
    print(phone_details)

In [ ]:
#
# # (1)Represent extracted data in flat file for onward/persistent ref.
#

# Extracted data
save_data_to_file('phone_details', phone_details_list) # main data dump file


# Extracted outlier data
outliers_combined_list = outlier_phone_details + outlier_price_details
if outliers_combined_list:
    # No point outputting an empty file 
    save_data_to_file('outlier_details', outliers_combined_list) # save potential anomalies separately for review

In [ ]:
#
# # (2)Represent data - Visualise some of caught/potential 'outliers'(hypothetical threshold examples)
#


max_records = 10 # Just to keep the output cleaner in notebook
table_data = tabulate(outliers_combined_list[:max_records], headers='keys', tablefmt='psql')

if len(table_data) > 0:
    # would add in other elements to print output / simplified here for clarity
    print("Potential (example)threshold/unexpected data found:\n -> Max Rating >{}\n -> Model Family Description >{}"
            .format(max_rating_value, max_model_family_length))
    print(table_data)

In [ ]:
#
# # (3)Represent data - start to consider/show simple stories within the data
#

visualise_average_rating_by_brand(phone_details_list)


visualise_upfront_prices(phone_details_list, 'Upfront Total Price')

visualise_upfront_prices(phone_details_list, 'Monthly Total Price')

In [ ]:
#
# # Hypothetical db store + extract (here done only in memory as db not portable)
#

# Create a database connection
conn = sqlite3.connect(":memory:")

# Call the create_phone_details_table() function to create the phone_details table
create_phone_details_table(conn)

# Call the insert_phone_details() function to insert the extracted phone details into the phone_details table
insert_phone_details(conn, phone_details_list)



# Execute a SELECT query on the phone_details table
c = conn.cursor()
c.execute("SELECT brand, model_family, rating_value, upfront_total_price, monthly_total_price FROM phone_details LIMIT 5")

# Fetch the results of the query and print them out
results = c.fetchall()
for row in results:
    print(row)

# Close the database connection
conn.close()